In [86]:
import json
import numpy as np
import matplotlib.pyplot as plt
import imageio
import os
from sklearn.decomposition import PCA

In [87]:
if not os.path.exists('temp_images'):
    os.makedirs('temp_images')

datos = []
with open('data.json', 'r') as f:
    for line in f:
        datos.append(json.loads(line))


In [88]:
qs = []
M_prevs = []
M_prev_contrs = []

for epoch_data in datos:
    qs.append(np.array(epoch_data['q']))
    M_prevs.append(np.array(epoch_data['M_prev']))
    M_prev_contrs.append(np.array(epoch_data['M_prev_contr']))

qs = np.array(qs)
M_prevs = np.array(M_prevs)
M_prev_contrs = np.array(M_prev_contrs)

In [89]:
comp_mat = np.vstack((qs, M_prevs, M_prev_contrs))

In [90]:
#2D
pca_comp_mat = PCA(n_components=2)
pca_2d = pca_comp_mat.fit_transform(comp_mat)

In [91]:
x_min, x_max = pca_2d[:, 0].min()-0.2, pca_2d[:, 0].max()+0.2
y_min, y_max = pca_2d[:, 1].min()-0.2, pca_2d[:, 1].max()+0.2

In [92]:
n = pca_2d.shape[0]
qs_2d = pca_2d[:n//3, :]
M_prevs_2d = pca_2d[n//3:2*n//3, :]
M_prev_contrs_2d = pca_2d[2*n//3:, :]

In [93]:
age = 0
epoch = 0
n_epochs = 20
for i in range(len(datos)):
    fig = plt.figure()
    ax = fig.add_subplot(111)
    
    ax.scatter(qs_2d[i, 0], qs_2d[i, 1], color='r', marker='o',  label='q')
    ax.scatter(M_prevs_2d[i, 0], M_prevs_2d[i, 1], color='g', marker='^', label='M_prev')
    ax.scatter(M_prev_contrs_2d[i, 0], M_prev_contrs_2d[i, 1], color='b', marker='s', label='M_prev_contr')
    
    if i%n_epochs == 0: 
        age = age+1
        epoch = 0
    else: 
        epoch = epoch+1

    ax.set_title(f'PCA - Age {age} Epoch {epoch}')
    ax.set_xlim([x_min, x_max])
    ax.set_ylim([y_min, y_max])
    ax.legend(loc='upper left', bbox_to_anchor=(1, 1))

    plt.savefig(f'temp_images/pca_{i}.png')
    plt.close()


In [94]:
images = []
for i in range(0, len(datos), 2):
    filename = f'temp_images/pca_{i}.png'
    images.append(imageio.v2.imread(filename))
        
imageio.mimsave('pca__2.gif', images, fps=2)
print("Done")

Done


In [85]:
for file_name in os.listdir('temp_images'):
    file_path = os.path.join('temp_images', file_name)
    if os.path.isfile(file_path):
        os.unlink(file_path)

os.rmdir('temp_images')